# Actuator - LEDs

In this notebook we will learn how to use the LED component to change the LED pattern on a Duckiebot.

In [ ]:
%load_ext autoreload
%autoreload 2

import os

# TODO: change this to the name of your Duckiebot
VEHICLE_NAME: str = "db21j3"

In [ ]:
# Actuator - LEDs - Set color and intensity

from duckietown.components.ros import LEDsPattern, LEDsDriverComponent

# define component
leds: LEDsDriverComponent = LEDsDriverComponent(vehicle_name=VEHICLE_NAME)
leds.start()

# define new pattern
intensity: float = 0.3
pattern: LEDsPattern = LEDsPattern(
    front_left=(0, 0, 0, intensity),
    front_right=(0, 0, 0, intensity),
    rear_right=(1, 0, 0, intensity),
    rear_left=(1, 0, 0, intensity),
)

# set pattern
leds.in_pattern.put(pattern)

# stop component
leds.stop()

In [ ]:
# Actuator - LEDs - Chameleon mode

import cv2
import numpy as np

from typing import Optional, Tuple
from functools import partial

from sklearn.cluster import KMeans
from matplotlib.colors import hsv_to_rgb, rgb_to_hsv

from duckietown.types import BGRImage, BGR8Color, BGRColor, IQueue, Queue
from duckietown.components.base import Component
from duckietown.components.ros import CameraDriverComponent, LEDsPattern, LEDsDriverComponent
from duckietown.components.rendering import ImageRendererComponent

OFF = (0, 0, 0, 0)

class MostDominantColorComponent(Component[BGRImage, BGRColor]):

    def __init__(self):
        super(MostDominantColorComponent, self).__init__()
        # queues
        self.in_bgr: IQueue = Queue()
        self.out_color: IQueue = Queue()

    def _crop(self, img: BGRImage) -> BGRImage:
        shape = img.shape
        w = int(shape[1]/2)
        h = int(shape[0]/2)
        x = shape[1]/2 - w/2
        y = shape[0]/2 - h/2
        return img[int(y):int(y+h), int(x):int(x+w)]

    def _most_dominant_color(self, img: BGRImage) -> BGRColor:
        # focus on 50% of the image around the center
        img = self._crop(img)
        pixels = img.reshape(-1, 3)
        # use k-means to cluster pixels
        kmeans = KMeans(n_clusters=1, n_init='auto')
        kmeans.fit(pixels)
        # most dominant color is the cluster center
        bgr: np.ndarray = kmeans.cluster_centers_[0]
        # in HSV, we can saturate S and V to enhance the color 
        bgr /= 255.
        rgb = bgr[::-1]
        hsv = rgb_to_hsv(rgb)
        hsv[1] = hsv[2] = 1.
        rgb = hsv_to_rgb(hsv)
        bgr = rgb[::-1]
        return bgr.tolist()
    
    def worker(self):
        while not self.is_shutdown:
            # get frame
            bgr: BGRImage = self.in_bgr.get()
            # find most dominant color
            color: BGRColor = self._most_dominant_color(bgr)
            # send color out
            self.out_color.put(color)

# define color-to-led-pattern component
class ColorToPatternComponent(Component[BGRColor, LEDsPattern]):

    def __init__(self):
        super(ColorToPatternComponent, self).__init__()
        # queues
        self.in_color: IQueue = Queue()
        self.out_pattern: IQueue = Queue()
    
    def worker(self):
        while not self.is_shutdown:
            # get color
            color: BGRColor = self.in_color.get()
            b, g, r = color
            # prepare pattern
            pattern: LEDsPattern = LEDsPattern(
                # low intensity on the front to avoid affecting the color of the object in front of the camera
                front_left=(r, g, b, 0.05),
                front_right=(r, g, b, 0.05),
                rear_right=(r, g, b, 1.0),
                rear_left=(r, g, b, 1.0),
            )
            # send pattern out
            self.out_pattern.put(pattern)

# define component
leds: LEDsDriverComponent = LEDsDriverComponent(vehicle_name=VEHICLE_NAME)
camera: CameraDriverComponent = CameraDriverComponent(vehicle_name=VEHICLE_NAME, throttle_rate=100)
renderer: ImageRendererComponent = ImageRendererComponent()
image2color: MostDominantColorComponent = MostDominantColorComponent()
color2pattern: ColorToPatternComponent = ColorToPatternComponent()

# connect components
renderer.in_image.wants(camera.out_bgr)
image2color.in_bgr.wants(camera.out_bgr)
color2pattern.in_color.wants(image2color.out_color)
leds.in_pattern.wants(color2pattern.out_pattern)

# start components
leds.start()
camera.start()
renderer.start()
image2color.start()
color2pattern.start()

# wait until the cell is stopped
renderer.join()

# stop components
leds.stop()
camera.stop()
renderer.stop()
image2color.stop()
color2pattern.stop()